In [ ]:
# %pip install spacy
# %pip install pdfminer pdfminer.six
%pip install PyPDF2 scikit-learn numpy

In [4]:
# import spacy

In [11]:
# nlp = spacy.load('de_core_news_md')

In [25]:
import os
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [26]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

In [27]:
def load_invoices(directory):
    invoices = {}
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            path = os.path.join(directory, filename)
            invoices[filename] = extract_text_from_pdf(path)
    return invoices

In [28]:
def calculate_similarity(input_text, database_texts):
    vectorizer = TfidfVectorizer()
    all_texts = [input_text] + list(database_texts.values())
    tfidf_matrix = vectorizer.fit_transform(all_texts)
    
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    return cosine_similarities[0]

In [31]:
os.getcwd()

'c:\\Users\\konda\\Documents\\intern-ass\\doc-sim'

In [35]:
def find_most_similar_invoice(input_invoice, database):
    input_text = extract_text_from_pdf(input_invoice)
    similarities = calculate_similarity(input_text, database)
    
    most_similar_index = np.argmax(similarities)
    most_similar_filename = list(database.keys())[most_similar_index]
    similarity_score = similarities[most_similar_index]
    
    return most_similar_filename, similarity_score

# Main execution
if __name__ == "__main__":
    train_directory = os.getcwd()+'/train'
    test_directory = os.getcwd()+'/test'
    
    # Load training invoices
    invoice_database = load_invoices(train_directory)
    
    # Process test invoices
    for test_invoice in os.listdir(test_directory):
        if test_invoice.endswith('.pdf'):
            test_invoice_path = os.path.join(test_directory, test_invoice)
            most_similar, score = find_most_similar_invoice(test_invoice_path, invoice_database)
            print(f"Test Invoice: {test_invoice}")
            print(f"Most Similar: {most_similar}")
            print(f"Similarity Score: {score:.4f}")
            print()

Test Invoice: invoice_102857.pdf
Most Similar: invoice_102856.pdf
Similarity Score: 0.7612

Test Invoice: invoice_77098.pdf
Most Similar: invoice_77073.pdf
Similarity Score: 0.8101

